# Chain of Thought (CoT) Prompting Tutorial

## Overview

This tutorial delves into Chain of Thought (CoT) prompting, an advanced technique in prompt engineering that guides AI models to dissect complex problems into detailed, step-by-step reasoning processes. We'll explore the implementation of CoT prompting using Amazon Nova via OpenRouter and LangChain.

## Motivation

With the rapid advancement of AI language models, there's a growing demand for outputs that are more transparent, logical, and verifiable. CoT prompting addresses this need by encouraging models to articulate their thought process, similar to how humans tackle complex problems. This approach not only enhances the accuracy of AI-generated responses but also increases their interpretability and reliability.

## Key Components

1. **CoT Prompting Fundamentals**: Introduction to the core concepts and basic implementation.
2. **Advanced CoT Strategies**: Exploration of more sophisticated CoT methodologies.
3. **Comparative Study**: Analysis of the distinctions between traditional and CoT prompting techniques.
4. **Practical Applications**: Utilizing CoT for a variety of complex problem-solving scenarios.

## Setup

Let's start by importing the necessary libraries and setting up our environment.

In [1]:
from os import getenv

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="bedrock/nova-lite-v1",
)

## Basic Chain of Thought Prompting

Let's start with a simple example to demonstrate the difference between a standard prompt and a Chain of Thought prompt.

In [2]:
# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question conciesly: {question}.",
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step conciesly: {question}",
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
60 km/h


Chain of Thought Response:
1. **Formula:** Average speed = Total distance / Total time

2. **Substitute:** Average speed = 120 km / 2 hours

3. **Calculate:** Average speed = 60 km/h



## Advanced Chain of Thought Techniques

Now, let's explore a more advanced CoT technique that encourages multi-step reasoning.

In [3]:
advanced_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Solve the following problem step by step. For each step:
1. State what you're going to calculate
2. Write the formula you'll use (if applicable)
3. Perform the calculation
4. Explain the result

Question: {question}

Solution:""",
)

advanced_cot_chain = advanced_cot_prompt | llm

complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

advanced_cot_response = advanced_cot_chain.invoke(complex_question).content
print(advanced_cot_response)

**Step 1: Calculate the time taken for the first part of the journey.**

1. **What we're calculating:** The time taken to travel the first 150 km.
2. **Formula:** Time = Distance / Speed
3. **Calculation:** Time1 = 150 km / 60 km/h = 2.5 hours
4. **Explanation:** It took 2.5 hours to travel the first 150 kilometers at a speed of 60 km/h.


**Step 2: Calculate the time taken for the second part of the journey.**

1. **What we're calculating:** The time taken to travel the second 100 km.
2. **Formula:** Time = Distance / Speed
3. **Calculation:** Time2 = 100 km / 50 km/h = 2 hours
4. **Explanation:** It took 2 hours to travel the second 100 kilometers at a speed of 50 km/h.


**Step 3: Calculate the total distance of the journey.**

1. **What we're calculating:** The total distance traveled.
2. **Calculation:** Total Distance = 150 km + 100 km = 250 km
4. **Explanation:** The car traveled a total of 250 kilometers.


**Step 4: Calculate the total time taken for the journey.**

1. **What 

## Auto Chain of Thought Techniques

Now, let's explore a technique that automatically generates CoT prompts.

In [4]:
reasoning_llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="google/gemini-2.0-flash-exp:free",
)

auto_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
Generate a general chain of thought prompt to solve the following question.
Please do not attempt to solve the question yourself.
Instead, generate a prompt that will help a language model to solve the question step by step.

Question: {question}

Chain of thought prompt:""",
)

auto_cot_chain = auto_cot_prompt | reasoning_llm

complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

auto_cot_response = auto_cot_chain.invoke(complex_question).content
print(auto_cot_response)

Okay, here's a chain-of-thought prompt designed to guide a language model through solving the average speed problem step-by-step:

**Prompt:**

"Let's solve this problem about average speed step-by-step. First, we need to break down the journey into individual segments.

1.  **Segment 1:** The car travels 150 km at 60 km/h. Calculate the time taken for this segment. Remember the relationship between distance, speed, and time.
2.  **Segment 2:** The car travels 100 km at 50 km/h. Calculate the time taken for this segment, using the same relationship as before.
3.  **Total Distance:** Now, calculate the total distance traveled by the car over the entire journey.
4.  **Total Time:** Next, calculate the total time taken for the entire journey by combining the times from Segment 1 and Segment 2.
5.  **Average Speed:** Finally, calculate the average speed for the entire journey. Remember, average speed is the total distance divided by the total time.
6.  **State the Final Answer:** Provide t

In [5]:
# 1. Keep existing auto_cot_chain
auto_cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="""Generate a general chain of thought prompt to solve the following question.
Please do not attempt to solve the question yourself.
Instead, generate a prompt that will help a language model to solve the question step by step.
Question: {question}
Chain of thought prompt:""",
)
auto_cot_chain = auto_cot_prompt | reasoning_llm

# 2. Modify advanced prompt to include generated CoT
advanced_cot_prompt = PromptTemplate(
    input_variables=["generated_cot", "question"],
    template="""
{generated_cot}
Question: {question}
Solution:""",
)

# 3. Create chained implementation
advanced_cot_chain = (
    RunnablePassthrough.assign(generated_cot=auto_cot_chain) | advanced_cot_prompt | llm
)

# 4. Invoke together
complex_question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"
response = advanced_cot_chain.invoke({"question": complex_question})
print(response.content)

Here's a solution following the steps outlined in the prompt:

**Step 1: Calculate the time taken for the first part of the journey.**

Time = Distance / Speed = 150 km / 60 km/h = 2.5 hours

**Step 2: Calculate the time taken for the second part of the journey.**

Time = Distance / Speed = 100 km / 50 km/h = 2 hours

**Step 3: Calculate the total distance traveled.**

Total distance = 150 km + 100 km = 250 km

**Step 4: Calculate the total time taken.**

Total time = 2.5 hours + 2 hours = 4.5 hours

**Step 5: Calculate the average speed.**

Average speed = Total distance / Total time = 250 km / 4.5 hours = 55.56 km/h (approximately)

Therefore, the average speed for the entire journey is approximately **55.56 km/h**.



## Comparative Analysis

Let's compare the effectiveness of standard prompting vs. CoT prompting on a more challenging problem.

In [7]:
challenging_question = """
A cylindrical water tank with a radius of 1.5 meters and a height of 4 meters is 2/3 full. 
If water is being added at a rate of 10 liters per minute, how long will it take for the tank to overflow? 
Give your answer in hours and minutes, rounded to the nearest minute. 
(Use 3.14159 for π and 1000 liters = 1 cubic meter)"""

standard_response = standard_chain.invoke(challenging_question).content
cot_response = advanced_cot_chain.invoke({"question": challenging_question}).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)

Standard Response:
1 hour and 26 minutes


Chain of Thought Response:
**1. Calculate the total volume of the cylindrical tank:**

* V = πr2h = 3.14159 * (1.5 m)2 * 4 m = 28.27431 cubic meters

**2. Calculate the current volume of water in the tank:**

* Current volume = (2/3) * 28.27431 cubic meters = 18.84954 cubic meters

**3. Calculate the remaining volume needed to fill the tank:**

* Remaining volume = 28.27431 cubic meters - 18.84954 cubic meters = 9.42477 cubic meters

**4. Convert the remaining volume to liters:**

* Remaining volume = 9.42477 cubic meters * 1000 liters/cubic meter = 9424.77 liters

**5. Calculate the time to fill the remaining volume in minutes:**

* Time = 9424.77 liters / 10 liters/minute = 942.477 minutes

**6. Convert the time to hours and minutes:**

* Hours = 942 minutes / 60 minutes/hour = 15.7078 hours
* Minutes = 0.7078 hours * 60 minutes/hour ≈ 42 minutes

**7. State your final answer:**

15 hours and 42 minutes

